In [ ]:
# Cell 1: Install required packages and setup
print(" Setting up Password Generator Environment...")

# Import required libraries
import random
import string
import secrets
import re
import json
from typing import List, Tuple, Dict
from datetime import datetime
import time

# Verify Python version
import sys
print(f" Python Version: {sys.version}")
print(f" All libraries imported successfully!")
print(" Environment ready for password generation!")

 Setting up Password Generator Environment...
 Python Version: 3.12.11 (main, Jun  4 2025, 08:56:18) [GCC 11.4.0]
 All libraries imported successfully!
 Environment ready for password generation!


In [ ]:
# Cell 2: Complete Password Generator Code - COPY ALL OF THIS
import random
import string
import secrets
import re
from typing import List, Tuple

class GooglePasswordGenerator:
    """
    Advanced Password generator following Google's password policies with comprehensive
    weak pattern detection using dictionary datasets:
    - At least 8 characters long (we'll default to 12-16 for better security)
    - Mix of uppercase and lowercase letters
    - At least one number
    - At least one special character
    - Avoid common passwords and patterns using extensive datasets
    - Avoid personal information patterns
    """

    def __init__(self, load_comprehensive_datasets=True):
        self.lowercase = string.ascii_lowercase
        self.uppercase = string.ascii_uppercase
        self.digits = string.digits
        self.special_chars = "!@#$%^&*()_+-=[]{}|;:,.<>?"

        # Load comprehensive weak pattern datasets
        if load_comprehensive_datasets:
            self._load_comprehensive_weak_datasets()
        else:
            self._load_basic_weak_patterns()

    def _load_basic_weak_patterns(self):
        """Load basic weak patterns (fallback)"""
        self.weak_patterns = [
            r'123+', r'abc+', r'qwerty', r'password', r'admin',
            r'(\w)\1{2,}',  # repeated characters
            r'(012|123|234|345|456|567|678|789|890)',  # sequential numbers
            r'(abc|bcd|cde|def|efg|fgh|ghi|hij|ijk|jkl|klm|lmn|mno|nop|opq|pqr|qrs|rst|stu|tuv|uvw|vwx|wxy|xyz)'  # sequential letters
        ]

        self.common_passwords = set([
            'password', '123456', 'password123', 'admin', 'qwerty', 'letmein',
            'welcome', 'monkey', '1234567890', 'abc123', 'Password1', 'iloveyou'
        ])

        self.dictionary_words = set([
            'password', 'welcome', 'computer', 'internet', 'security', 'login',
            'system', 'manager', 'service', 'access', 'network', 'database'
        ])

        self.keyboard_patterns = [
            'qwerty', 'asdf', 'zxcv', '1234', 'abcd', 'qwertyuiop',
            'asdfghjkl', 'zxcvbnm', '1234567890'
        ]

        print("📊 Loaded basic weak patterns dataset")

    def _load_comprehensive_weak_datasets(self):
        """
        Load comprehensive datasets for weak pattern detection
        In production, these would be loaded from external files/databases
        """
        # Top 10,000 most common passwords (sample from real breach data)
        self.common_passwords = set([
            # From HaveIBeenPwned and other breach databases
            'password', '123456', 'password123', 'admin', 'qwerty', 'letmein',
            'welcome', 'monkey', '1234567890', 'abc123', 'Password1', 'iloveyou',
            'princess', 'rockyou', 'football', 'baseball', 'welcome123',
            'sunshine', 'master', 'shadow', 'ashley', 'jesus', 'michael',
            'ninja', 'mustang', 'password1', 'password!', 'superman', 'batman',
            'trustno1', 'hello123', 'dragon', 'passw0rd', 'killer', 'harley',
            'jordan23', 'jennifer', 'hunter', '2000', 'test', 'batman',
            'thomas', 'robert', 'john', 'guest', 'andrea', 'cheese',
            'michelle', 'love', 'oracle', 'merlin', 'jessica', 'purple',
            'secret', 'hannah', 'orange', 'nothing', 'garden', 'freedom',
            'whatever', 'nicole', 'chicken', 'friday', 'pepper', 'maverick',
            'ferrari', 'sandwich', 'coffee', 'computer', 'internet', 'startrek'
        ])

        # Common dictionary words that appear in passwords
        self.dictionary_words = set([
            'password', 'welcome', 'computer', 'internet', 'security', 'login',
            'system', 'manager', 'service', 'access', 'network', 'database',
            'server', 'office', 'company', 'business', 'website', 'email',
            'account', 'username', 'default', 'temporary', 'change', 'reset',
            'public', 'private', 'secret', 'hidden', 'secure', 'protected',
            'family', 'friend', 'mother', 'father', 'sister', 'brother',
            'spring', 'summer', 'winter', 'autumn', 'january', 'february',
            'march', 'april', 'monday', 'tuesday', 'wednesday', 'thursday',
            'friday', 'saturday', 'sunday', 'morning', 'evening', 'night'
        ])

        # Keyboard patterns and sequences
        self.keyboard_patterns = [
            # QWERTY patterns
            'qwerty', 'qwertyui', 'qwertyuiop', 'asdf', 'asdfgh', 'asdfghjkl',
            'zxcv', 'zxcvbn', 'zxcvbnm', 'qwer', 'wert', 'erty', 'yuiop',
            'sdfg', 'dfgh', 'fghj', 'ghjk', 'hjkl', 'xcvb', 'cvbn', 'vbnm',
            # Number patterns
            '1234', '12345', '123456', '1234567', '12345678', '123456789', '1234567890',
            '0987', '09876', '098765', '0987654', '09876543', '098765432', '0987654321',
            # Alphabet sequences
            'abc', 'abcd', 'abcde', 'abcdef', 'abcdefg', 'abcdefgh', 'abcdefghi',
            'xyz', 'wxyz', 'vwxyz', 'uvwxyz', 'tuvwxyz', 'stuvwxyz'
        ]

        # Common substitution patterns (l33t speak)
        self.leet_patterns = {
            'a': ['@', '4'], 'e': ['3'], 'i': ['1', '!'], 'o': ['0'],
            's': ['$', '5'], 't': ['7'], 'l': ['1'], 'g': ['6', '9']
        }

        # Date patterns (years, months, days)
        self.date_patterns = [
            # Years
            r'19\d{2}', r'20\d{2}', r'21\d{2}',
            # Month/day combinations
            r'(0[1-9]|1[0-2])(0[1-9]|[12]\d|3[01])',
            r'(jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec)\d{2,4}',
            # Common date formats
            r'\d{1,2}/\d{1,2}/\d{2,4}', r'\d{1,2}-\d{1,2}-\d{2,4}'
        ]

        # Personal information patterns
        self.personal_patterns = [
            r'(mom|dad|son|daughter|wife|husband)\d*',
            r'(name|surname|lastname|firstname)\d*',
            r'(home|house|address|phone|mobile)\d*',
            r'(birthday|anniversary|wedding)\d*'
        ]

        # Enhanced regex patterns for comprehensive detection
        self.weak_patterns = [
            # Original basic patterns
            r'(\w)\1{2,}',  # repeated characters (aaa, 111, etc.)
            r'(012|123|234|345|456|567|678|789|890|987|876|765|654|543|432|321|210)',
            r'(abc|bcd|cde|def|efg|fgh|ghi|hij|ijk|jkl|klm|lmn|mno|nop|opq|pqr|qrs|rst|stu|tuv|uvw|vwx|wxy|xyz)',

            # Enhanced patterns
            r'(.{1,3})\1{2,}',  # repeated sequences (abcabc, 123123)
            r'(password|admin|login|user|guest|test|demo)',  # common words
            r'\d{4,}',  # long number sequences
            r'[a-z]{5,}',  # long lowercase sequences
            r'[A-Z]{3,}',  # long uppercase sequences

            # Keyboard walking patterns
            r'(qwe|wer|ert|rty|tyu|yui|uio|iop|asd|sdf|dfg|fgh|ghj|hjk|jkl|zxc|xcv|cvb|vbn|bnm)',
            r'(poi|oir|rew|ewq|lkj|kjh|jhg|gfd|fds|dsa|mnb|nbv|vcx|cxz)',

            # Phone number patterns
            r'\d{3}-?\d{3}-?\d{4}',  # xxx-xxx-xxxx or xxxxxxxxxx
            r'\(\d{3}\)\s?\d{3}-?\d{4}',  # (xxx) xxx-xxxx
        ]

        print(" Loaded comprehensive weak patterns dataset:")
        print(f"   • {len(self.common_passwords):,} common passwords")
        print(f"   • {len(self.dictionary_words)} dictionary words")
        print(f"   • {len(self.keyboard_patterns)} keyboard patterns")
        print(f"   • {len(self.weak_patterns)} regex patterns")
        print(f"   • {len(self.date_patterns)} date patterns")
        print(f"   • Enhanced l33t speak detection")

    def generate_secure_password(self, length: int = 14, exclude_ambiguous: bool = True) -> str:
        """
        Generate a secure password following Google's policies

        Args:
            length: Password length (minimum 8, recommended 12+)
            exclude_ambiguous: Exclude ambiguous characters like 0, O, l, 1

        Returns:
            Generated secure password
        """
        if length < 8:
            raise ValueError("Password length must be at least 8 characters")

        # Prepare character sets
        chars = self._prepare_character_sets(exclude_ambiguous)

        # Generate password with guaranteed character diversity
        password = self._generate_with_requirements(chars, length)

        # Validate and regenerate if needed
        max_attempts = 10
        attempts = 0

        while not self._is_password_strong(password) and attempts < max_attempts:
            password = self._generate_with_requirements(chars, length)
            attempts += 1

        return password

    def _prepare_character_sets(self, exclude_ambiguous: bool) -> dict:
        """Prepare character sets based on preferences"""
        chars = {
            'lowercase': self.lowercase,
            'uppercase': self.uppercase,
            'digits': self.digits,
            'special': self.special_chars
        }

        if exclude_ambiguous:
            # Remove ambiguous characters
            chars['lowercase'] = chars['lowercase'].replace('l', '')
            chars['uppercase'] = chars['uppercase'].replace('O', '').replace('I', '')
            chars['digits'] = chars['digits'].replace('0', '').replace('1', '')
            chars['special'] = chars['special'].replace('|', '')

        return chars

    def _generate_with_requirements(self, chars: dict, length: int) -> str:
        """Generate password ensuring all character types are included"""
        password_chars = []

        # Ensure at least one character from each required category
        password_chars.append(secrets.choice(chars['lowercase']))
        password_chars.append(secrets.choice(chars['uppercase']))
        password_chars.append(secrets.choice(chars['digits']))
        password_chars.append(secrets.choice(chars['special']))

        # Fill remaining length with random characters from all sets
        all_chars = ''.join(chars.values())
        for _ in range(length - 4):
            password_chars.append(secrets.choice(all_chars))

        # Shuffle the password to avoid predictable patterns
        secrets.SystemRandom().shuffle(password_chars)

        return ''.join(password_chars)

    def _is_password_strong(self, password: str) -> bool:
        """Enhanced password strength validation using comprehensive datasets"""
        password_lower = password.lower()

        # Check against common passwords database
        if self._check_common_passwords(password_lower):
            return False

        # Check against dictionary words
        if self._check_dictionary_words(password_lower):
            return False

        # Check keyboard patterns
        if self._check_keyboard_patterns(password_lower):
            return False

        # Check l33t speak variations
        if self._check_leet_variations(password_lower):
            return False

        # Check date patterns
        if self._check_date_patterns(password):
            return False

        # Check personal information patterns
        if self._check_personal_patterns(password_lower):
            return False

        # Check regex patterns
        for pattern in self.weak_patterns:
            if re.search(pattern, password_lower, re.IGNORECASE):
                return False

        # Check for repeated segments
        if self._has_repeated_segments(password):
            return False

        return True

    def _check_common_passwords(self, password: str) -> bool:
        """Check against common passwords dataset"""
        # Direct match
        if password in self.common_passwords:
            return True

        # Check with common variations
        variations = [
            password + '1', password + '123', password + '!',
            '1' + password, '123' + password, '!' + password,
            password.capitalize(), password.upper()
        ]

        for variation in variations:
            if variation.lower() in self.common_passwords:
                return True

        return False

    def _check_dictionary_words(self, password: str) -> bool:
        """Check for dictionary words with minimal modifications"""
        # Check if password is mostly a dictionary word
        for word in self.dictionary_words:
            if len(word) >= 4:  # Only check meaningful words
                # Direct match
                if word in password:
                    # If dictionary word takes up >60% of password, reject
                    if len(word) / len(password) > 0.6:
                        return True

                # Check with number suffixes
                if password.startswith(word) and password[len(word):].isdigit():
                    return True

                # Check with number prefixes
                if password.endswith(word) and password[:-len(word)].isdigit():
                    return True

        return False

    def _check_keyboard_patterns(self, password: str) -> bool:
        """Check for keyboard walking patterns"""
        for pattern in self.keyboard_patterns:
            if pattern in password:
                # If keyboard pattern takes up >50% of password, reject
                if len(pattern) / len(password) > 0.5:
                    return True

            # Check reverse patterns
            if pattern[::-1] in password:
                if len(pattern) / len(password) > 0.5:
                    return True

        return False

    def _check_leet_variations(self, password: str) -> bool:
        """Check for simple l33t speak variations of common passwords"""
        # Convert l33t back to normal letters
        normalized = password
        for letter, substitutes in self.leet_patterns.items():
            for substitute in substitutes:
                normalized = normalized.replace(substitute, letter)

        # Check if normalized version is a common password
        return normalized in self.common_passwords

    def _check_date_patterns(self, password: str) -> bool:
        """Check for date patterns in passwords"""
        for pattern in self.date_patterns:
            if re.search(pattern, password):
                return True
        return False

    def _check_personal_patterns(self, password: str) -> bool:
        """Check for personal information patterns"""
        for pattern in self.personal_patterns:
            if re.search(pattern, password):
                return True
        return False

    def get_detailed_weakness_analysis(self, password: str) -> dict:
        """
        Detailed analysis of what specific weaknesses were found
        This is useful for understanding why a password was rejected
        """
        password_lower = password.lower()
        weaknesses_found = {
            'common_passwords': [],
            'dictionary_words': [],
            'keyboard_patterns': [],
            'leet_variations': [],
            'date_patterns': [],
            'personal_patterns': [],
            'regex_patterns': [],
            'repeated_segments': []
        }

        # Check each category and record what was found

        # Common passwords
        if password_lower in self.common_passwords:
            weaknesses_found['common_passwords'].append(f"'{password}' is a commonly used password")

        variations = [password + '1', password + '123', password + '!']
        for var in variations:
            if var.lower() in self.common_passwords:
                weaknesses_found['common_passwords'].append(f"Variation '{var}' is commonly used")

        # Dictionary words
        for word in self.dictionary_words:
            if len(word) >= 4 and word in password_lower:
                if len(word) / len(password) > 0.6:
                    weaknesses_found['dictionary_words'].append(f"Contains dictionary word: '{word}'")

        # Keyboard patterns
        for pattern in self.keyboard_patterns:
            if pattern in password_lower and len(pattern) / len(password) > 0.5:
                weaknesses_found['keyboard_patterns'].append(f"Contains keyboard pattern: '{pattern}'")

        # Date patterns
        for pattern in self.date_patterns:
            matches = re.findall(pattern, password)
            if matches:
                weaknesses_found['date_patterns'].extend([f"Date pattern found: '{match}'" for match in matches])

        # Regex patterns
        for pattern in self.weak_patterns:
            matches = re.findall(pattern, password_lower, re.IGNORECASE)
            if matches:
                weaknesses_found['regex_patterns'].extend([f"Weak pattern: '{match}'" for match in matches])

        # Repeated segments
        if self._has_repeated_segments(password):
            weaknesses_found['repeated_segments'].append("Contains repeated character segments")

        return weaknesses_found

    def _has_repeated_segments(self, password: str, min_segment_length: int = 3) -> bool:
        """Check for repeated segments in password"""
        for i in range(len(password) - min_segment_length):
            segment = password[i:i + min_segment_length]
            if password.count(segment) > 1:
                return True
        return False

    def analyze_password_strength(self, password: str) -> dict:
        """
        Comprehensive password strength analysis

        Returns:
            Dictionary with detailed analysis including strengths, weaknesses,
            policies met, and overall score
        """
        analysis = {
            'password': password,
            'length': len(password),
            'strengths': [],
            'weaknesses': [],
            'policies_met': [],
            'policies_failed': [],
            'character_breakdown': {},
            'security_score': 0,
            'strength_level': '',
            'recommendations': []
        }

        # Character breakdown
        lowercase_count = sum(1 for c in password if c.islower())
        uppercase_count = sum(1 for c in password if c.isupper())
        digit_count = sum(1 for c in password if c.isdigit())
        special_count = sum(1 for c in password if c in self.special_chars)

        analysis['character_breakdown'] = {
            'lowercase': lowercase_count,
            'uppercase': uppercase_count,
            'digits': digit_count,
            'special_chars': special_count,
            'total_chars': len(password)
        }

        # Policy checks and scoring
        score = 0

        # Length policy (Google: minimum 8 characters)
        if len(password) >= 8:
            analysis['policies_met'].append("✓ Minimum 8 characters required")
            if len(password) >= 12:
                analysis['strengths'].append("Excellent length (12+ characters)")
                score += 25
            elif len(password) >= 10:
                analysis['strengths'].append("Good length (10+ characters)")
                score += 20
            else:
                analysis['strengths'].append("Meets minimum length requirement")
                score += 15
        else:
            analysis['policies_failed'].append("✗ Must be at least 8 characters")
            analysis['weaknesses'].append("Password too short")

        # Lowercase letters policy
        if lowercase_count > 0:
            analysis['policies_met'].append("✓ Contains lowercase letters")
            if lowercase_count >= 3:
                analysis['strengths'].append("Multiple lowercase letters")
                score += 10
            else:
                score += 8
        else:
            analysis['policies_failed'].append("✗ Must contain lowercase letters")
            analysis['weaknesses'].append("Missing lowercase letters")

        # Uppercase letters policy
        if uppercase_count > 0:
            analysis['policies_met'].append("✓ Contains uppercase letters")
            if uppercase_count >= 2:
                analysis['strengths'].append("Multiple uppercase letters")
                score += 10
            else:
                score += 8
        else:
            analysis['policies_failed'].append("✗ Must contain uppercase letters")
            analysis['weaknesses'].append("Missing uppercase letters")

        # Numbers policy
        if digit_count > 0:
            analysis['policies_met'].append("✓ Contains numbers")
            if digit_count >= 2:
                analysis['strengths'].append("Multiple numbers for better security")
                score += 10
            else:
                score += 8
        else:
            analysis['policies_failed'].append("✗ Must contain at least one number")
            analysis['weaknesses'].append("Missing numbers")

        # Special characters policy
        if special_count > 0:
            analysis['policies_met'].append("✓ Contains special characters")
            if special_count >= 2:
                analysis['strengths'].append("Multiple special characters")
                score += 15
            else:
                analysis['strengths'].append("Contains special characters")
                score += 12
        else:
            analysis['policies_failed'].append("✗ Must contain special characters")
            analysis['weaknesses'].append("Missing special characters")

        # Advanced security checks with dataset integration
        weakness_details = self.get_detailed_weakness_analysis(password)
        total_weaknesses = sum(len(v) for v in weakness_details.values())

        if total_weaknesses == 0:
            analysis['strengths'].append("Passed comprehensive security screening")
            score += 15
        else:
            analysis['strengths'].append("Basic security requirements met")

        if not self._has_common_patterns(password):
            analysis['strengths'].append("No common weak patterns detected")
            score += 10
        else:
            # Add specific weakness details
            for category, issues in weakness_details.items():
                if issues:
                    category_name = category.replace('_', ' ').title()
                    analysis['weaknesses'].append(f"{category_name}: {issues[0]}")
                    score -= 8  # Penalize each type of weakness

        if not self._has_repeated_segments(password):
            analysis['strengths'].append("No repetitive segments")
            score += 8
        else:
            analysis['weaknesses'].append("Contains repeated segments")
            score -= 10

        if self._check_entropy(password):
            analysis['strengths'].append("High entropy (randomness)")
            score += 12

        # Calculate final score and strength level
        analysis['security_score'] = max(0, min(100, score))

        if analysis['security_score'] >= 90:
            analysis['strength_level'] = "EXCELLENT"
        elif analysis['security_score'] >= 80:
            analysis['strength_level'] = "VERY STRONG"
        elif analysis['security_score'] >= 70:
            analysis['strength_level'] = "STRONG"
        elif analysis['security_score'] >= 60:
            analysis['strength_level'] = "MODERATE"
        elif analysis['security_score'] >= 40:
            analysis['strength_level'] = "WEAK"
        else:
            analysis['strength_level'] = "VERY WEAK"

        # Generate recommendations
        analysis['recommendations'] = self._generate_recommendations(analysis)

        return analysis

    def _check_character_diversity(self, password: str) -> bool:
        """Check if password has good character diversity"""
        char_types = 0
        if any(c.islower() for c in password):
            char_types += 1
        if any(c.isupper() for c in password):
            char_types += 1
        if any(c.isdigit() for c in password):
            char_types += 1
        if any(c in self.special_chars for c in password):
            char_types += 1

        return char_types == 4 and len(set(password)) / len(password) > 0.7

    def _has_common_patterns(self, password: str) -> bool:
        """Check for common weak patterns"""
        return not self._is_password_strong(password)

    def _check_entropy(self, password: str) -> bool:
        """Basic entropy check"""
        unique_chars = len(set(password))
        return unique_chars / len(password) > 0.8 and unique_chars >= 8

    def _generate_recommendations(self, analysis: dict) -> List[str]:
        """Generate personalized recommendations based on analysis"""
        recommendations = []

        if analysis['security_score'] < 70:
            if analysis['length'] < 12:
                recommendations.append("Increase password length to 12+ characters")
            if analysis['character_breakdown']['special_chars'] < 2:
                recommendations.append("Add more special characters (!@#$%^&*)")
            if analysis['character_breakdown']['digits'] < 2:
                recommendations.append("Include multiple numbers")

        if "Contains common patterns" in analysis['weaknesses']:
            recommendations.append("Avoid sequential characters or common patterns")

        if "Contains repeated segments" in analysis['weaknesses']:
            recommendations.append("Remove repetitive character sequences")

        if not recommendations:
            recommendations.append("Password meets all security requirements!")

        return recommendations

    def generate_multiple_passwords(self, count: int = 5, length: int = 14) -> List[dict]:
        """Generate multiple password options with full analysis"""
        passwords = []
        for _ in range(count):
            password = self.generate_secure_password(length)
            analysis = self.analyze_password_strength(password)
            passwords.append(analysis)
        return passwords

    def check_password_strength(self, password: str) -> Tuple[bool, List[str]]:
        """
        Legacy method for backward compatibility
        Check if a password meets Google's requirements

        Returns:
            (is_strong, list_of_issues)
        """
        analysis = self.analyze_password_strength(password)
        return len(analysis['policies_failed']) == 0, analysis['policies_failed']

    def generate_passphrase(self, word_count: int = 4, separator: str = "-") -> str:
        """
        Generate a passphrase-style password (alternative approach)
        Uses random words with numbers and special characters
        """
        # Simple word list (in practice, you'd use a larger dictionary)
        words = [
            "apple", "ocean", "guitar", "mountain", "wizard", "dragon", "sunset", "river",
            "castle", "forest", "thunder", "crystal", "phoenix", "meadow", "starlight",
            "butterfly", "rainbow", "volcano", "compass", "lighthouse", "adventure",
            "treasure", "harmony", "mystery", "whisper", "journey", "discovery", "magic"
        ]

        selected_words = [secrets.choice(words).capitalize() for _ in range(word_count)]

        # Add numbers and special characters
        passphrase = separator.join(selected_words)
        passphrase += str(secrets.randbelow(100) + 100)  # Add random number
        passphrase += secrets.choice(self.special_chars)

        return passphrase

# Example usage and testing
def demo_password_generator():
    """Demonstrate the password generator functionality with detailed analysis"""
    generator = GooglePasswordGenerator()

    print("=" * 80)
    print(" GOOGLE-COMPLIANT PASSWORD GENERATOR WITH STRENGTH ANALYSIS 🔐")
    print("=" * 80)

    # Generate single password with full analysis
    print("\n1.  SINGLE PASSWORD GENERATION WITH FULL ANALYSIS:")
    print("-" * 60)
    password = generator.generate_secure_password(length=16)
    analysis = generator.analyze_password_strength(password)

    print(f" Generated Password: {analysis['password']}")
    print(f" Length: {analysis['length']} characters")
    print(f" Security Score: {analysis['security_score']}/100")
    print(f" Strength Level: {analysis['strength_level']}")

    print(f"\n Character Breakdown:")
    breakdown = analysis['character_breakdown']
    print(f"  • Lowercase: {breakdown['lowercase']} characters")
    print(f"  • Uppercase: {breakdown['uppercase']} characters")
    print(f"  • Numbers: {breakdown['digits']} characters")
    print(f"  • Special: {breakdown['special_chars']} characters")

    print(f"\n Google Policies Met ({len(analysis['policies_met'])}):")
    for policy in analysis['policies_met']:
        print(f"  {policy}")

    if analysis['policies_failed']:
        print(f"\n Policies Failed ({len(analysis['policies_failed'])}):")
        for policy in analysis['policies_failed']:
            print(f"  {policy}")

    print(f"\n Password Strengths ({len(analysis['strengths'])}):")
    for strength in analysis['strengths']:
        print(f"  • {strength}")

    if analysis['weaknesses']:
        print(f"\n Areas for Improvement ({len(analysis['weaknesses'])}):")
        for weakness in analysis['weaknesses']:
            print(f"  • {weakness}")

    print(f"\n")

In [ ]:
# Cell 3: Basic Functionality Test
print(" BASIC FUNCTIONALITY TEST")
print("=" * 50)

# Initialize the generator
generator = GooglePasswordGenerator(load_comprehensive_datasets=True)

# Generate a single password
print("\n1. Single Password Generation:")
password = generator.generate_secure_password(length=16)
print(f"Generated: {password}")

# Quick strength check
analysis = generator.analyze_password_strength(password)
print(f"Score: {analysis['security_score']}/100")
print(f"Level: {analysis['strength_level']}")

print("\n Basic functionality working!")

 BASIC FUNCTIONALITY TEST
 Loaded comprehensive weak patterns dataset:
   • 70 common passwords
   • 54 dictionary words
   • 48 keyboard patterns
   • 12 regex patterns
   • 7 date patterns
   • Enhanced l33t speak detection

1. Single Password Generation:
Generated: W-ee98&AF>$Q:]C=
Score: 100/100
Level: EXCELLENT

 Basic functionality working!


In [ ]:
# Cell 4: Full Demo
print(" RUNNING COMPLETE PASSWORD GENERATOR DEMO")
print("This may take a few seconds...")

# Run the complete demo
demo_password_generator()

 RUNNING COMPLETE PASSWORD GENERATOR DEMO
This may take a few seconds...
 Loaded comprehensive weak patterns dataset:
   • 70 common passwords
   • 54 dictionary words
   • 48 keyboard patterns
   • 12 regex patterns
   • 7 date patterns
   • Enhanced l33t speak detection
 GOOGLE-COMPLIANT PASSWORD GENERATOR WITH STRENGTH ANALYSIS 🔐

1.  SINGLE PASSWORD GENERATION WITH FULL ANALYSIS:
------------------------------------------------------------
 Generated Password: b]F$6]Ko;!z*Hd]Z
 Length: 16 characters
 Security Score: 100/100
 Strength Level: EXCELLENT

 Character Breakdown:
  • Lowercase: 4 characters
  • Uppercase: 4 characters
  • Numbers: 1 characters
  • Special: 7 characters

 Google Policies Met (5):
  ✓ Minimum 8 characters required
  ✓ Contains lowercase letters
  ✓ Contains uppercase letters
  ✓ Contains numbers
  ✓ Contains special characters

 Password Strengths (8):
  • Excellent length (12+ characters)
  • Multiple lowercase letters
  • Multiple uppercase letters
  • Mu

In [ ]:
# Cell 5: Web Integration Functions
def quick_generate_password(length=14, show_analysis=True):
    """Quick password generation with analysis"""
    generator = GooglePasswordGenerator()
    password = generator.generate_secure_password(length)

    if show_analysis:
        analysis = generator.analyze_password_strength(password)
        print(f" Password: {password}")
        print(f" Score: {analysis['security_score']}/100 ({analysis['strength_level']})")
        return password, analysis
    else:
        return password

def batch_generate_passwords(count=5, length=14):
    """Generate multiple passwords for comparison"""
    generator = GooglePasswordGenerator()
    passwords = generator.generate_multiple_passwords(count, length)

    print(f" Generated {count} passwords:")
    for i, analysis in enumerate(passwords, 1):
        print(f"{i}. {analysis['password']} (Score: {analysis['security_score']}/100)")

    return passwords

def validate_user_password(password):
    """Validate a user-provided password"""
    generator = GooglePasswordGenerator()
    analysis = generator.analyze_password_strength(password)

    print(f" Analyzing: '{password}'")
    print(f" Score: {analysis['security_score']}/100")
    print(f" Level: {analysis['strength_level']}")

    if analysis['policies_failed']:
        print(" Failed Policies:")
        for policy in analysis['policies_failed']:
            print(f"   {policy}")

    if analysis['recommendations']:
        print(" Recommendations:")
        for rec in analysis['recommendations']:
            print(f"   • {rec}")

    return analysis

print(" Web integration functions ready!")

 Web integration functions ready!


In [ ]:
# Cell 6: Test Interactive Functions
print(" TESTING INTERACTIVE FUNCTIONS")
print("=" * 50)

# Test 1: Quick generation
print("\n1. Quick Password Generation:")
password, analysis = quick_generate_password(length=16)

# Test 2: Batch generation
print("\n2. Batch Generation:")
batch_passwords = batch_generate_passwords(count=3, length=12)

# Test 3: Validation
print("\n3. Password Validation Test:")
validate_user_password("weakpassword")
print()
validate_user_password("MyStr0ng@P@ssw0rd2024!")

 TESTING INTERACTIVE FUNCTIONS

1. Quick Password Generation:
 Loaded comprehensive weak patterns dataset:
   • 70 common passwords
   • 54 dictionary words
   • 48 keyboard patterns
   • 12 regex patterns
   • 7 date patterns
   • Enhanced l33t speak detection
 Password: XP{RQ67[TC;8Y9h;
 Score: 100/100 (EXCELLENT)

2. Batch Generation:
 Loaded comprehensive weak patterns dataset:
   • 70 common passwords
   • 54 dictionary words
   • 48 keyboard patterns
   • 12 regex patterns
   • 7 date patterns
   • Enhanced l33t speak detection
 Generated 3 passwords:
1. C2Vy!jn-,;dX (Score: 100/100)
2. &5e8Gw;T33HR (Score: 100/100)
3. EV&9@{ps[U]J (Score: 100/100)

3. Password Validation Test:
 Loaded comprehensive weak patterns dataset:
   • 70 common passwords
   • 54 dictionary words
   • 48 keyboard patterns
   • 12 regex patterns
   • 7 date patterns
   • Enhanced l33t speak detection
 Analyzing: 'weakpassword'
 Score: 27/100
 Level: VERY WEAK
 Failed Policies:
   ✗ Must contain uppercase l

{'password': 'MyStr0ng@P@ssw0rd2024!',
 'length': 22,
 'strengths': ['Excellent length (12+ characters)',
  'Multiple lowercase letters',
  'Multiple uppercase letters',
  'Multiple numbers for better security',
  'Multiple special characters',
  'Basic security requirements met',
  'No repetitive segments'],
 'weaknesses': ["Date Patterns: Date pattern found: '2024'",
  "Regex Patterns: Weak pattern: '2024'"],
 'policies_met': ['✓ Minimum 8 characters required',
  '✓ Contains lowercase letters',
  '✓ Contains uppercase letters',
  '✓ Contains numbers',
  '✓ Contains special characters'],
 'policies_failed': [],
 'character_breakdown': {'lowercase': 10,
  'uppercase': 3,
  'digits': 6,
  'special_chars': 3,
  'total_chars': 22},
 'security_score': 62,
 'strength_level': 'MODERATE',
 'recommendations': ['Password meets all security requirements!']}

In [ ]:
# Cell 7: Production API Functions
class PasswordAPI:
    """Production-ready API wrapper for password generation"""

    def __init__(self):
        self.generator = GooglePasswordGenerator()

    def generate_password_api(self, length=14, count=1, exclude_ambiguous=True):
        """API endpoint for password generation"""
        try:
            if count == 1:
                password = self.generator.generate_secure_password(
                    length=length,
                    exclude_ambiguous=exclude_ambiguous
                )
                analysis = self.generator.analyze_password_strength(password)
                return {
                    "success": True,
                    "password": password,
                    "analysis": analysis,
                    "timestamp": datetime.now().isoformat()
                }
            else:
                passwords = self.generator.generate_multiple_passwords(count, length)
                return {
                    "success": True,
                    "passwords": passwords,
                    "count": len(passwords),
                    "timestamp": datetime.now().isoformat()
                }
        except Exception as e:
            return {
                "success": False,
                "error": str(e),
                "timestamp": datetime.now().isoformat()
            }

    def validate_password_api(self, password):
        """API endpoint for password validation"""
        try:
            analysis = self.generator.analyze_password_strength(password)
            weakness_details = self.generator.get_detailed_weakness_analysis(password)

            return {
                "success": True,
                "password": password,
                "is_strong": len(analysis['policies_failed']) == 0,
                "security_score": analysis['security_score'],
                "strength_level": analysis['strength_level'],
                "policies_met": analysis['policies_met'],
                "policies_failed": analysis['policies_failed'],
                "strengths": analysis['strengths'],
                "weaknesses": analysis['weaknesses'],
                "recommendations": analysis['recommendations'],
                "weakness_details": weakness_details,
                "character_breakdown": analysis['character_breakdown'],
                "timestamp": datetime.now().isoformat()
            }
        except Exception as e:
            return {
                "success": False,
                "error": str(e),
                "timestamp": datetime.now().isoformat()
            }

# Initialize API
api = PasswordAPI()
print(" Production API ready!")

# Test API functions
print("\n Testing API Functions:")
print("1. Generate Password API:")
result = api.generate_password_api(length=16)
print(f"   Success: {result['success']}")
print(f"   Password: {result['password']}")
print(f"   Score: {result['analysis']['security_score']}/100")

print("\n2. Validate Password API:")
validation = api.validate_password_api("TestP@ssw0rd123!")
print(f"   Success: {validation['success']}")
print(f"   Strong: {validation['is_strong']}")
print(f"   Score: {validation['security_score']}/100")

 Loaded comprehensive weak patterns dataset:
   • 70 common passwords
   • 54 dictionary words
   • 48 keyboard patterns
   • 12 regex patterns
   • 7 date patterns
   • Enhanced l33t speak detection
 Production API ready!

 Testing API Functions:
1. Generate Password API:
   Success: True
   Password: {*Z43*?)]k#WG*:S
   Score: 100/100

2. Validate Password API:
   Success: True
   Strong: True
   Score: 82/100


In [ ]:
# Cell 8: Save and Export Functions
def export_sample_passwords(count=10, filename="sample_passwords.json"):
    """Export sample passwords for testing"""
    api = PasswordAPI()

    sample_data = {
        "generated_at": datetime.now().isoformat(),
        "total_passwords": count,
        "passwords": []
    }

    for i in range(count):
        result = api.generate_password_api(length=random.randint(12, 18))
        if result['success']:
            sample_data["passwords"].append({
                "id": i + 1,
                "password": result['password'],
                "length": len(result['password']),
                "score": result['analysis']['security_score'],
                "level": result['analysis']['strength_level']
            })

    # Save to JSON (in Colab, this creates a downloadable file)
    with open(filename, 'w') as f:
        json.dump(sample_data, f, indent=2)

    print(f" Exported {count} sample passwords to {filename}")
    print(" File available for download in Colab Files panel")

    return sample_data

def create_integration_template():
    """Create template code for website integration"""
    template = '''
# Website Integration Template
# Copy this code to integrate with your web application

from your_password_generator import PasswordAPI

# Initialize API
password_api = PasswordAPI()

# Generate password endpoint
@app.route('/api/generate-password', methods=['POST'])
def generate_password():
    length = request.json.get('length', 14)
    count = request.json.get('count', 1)

    result = password_api.generate_password_api(length=length, count=count)
    return jsonify(result)

# Validate password endpoint
@app.route('/api/validate-password', methods=['POST'])
def validate_password():
    password = request.json.get('password', '')

    result = password_api.validate_password_api(password)
    return jsonify(result)
'''

    with open('website_integration_template.py', 'w') as f:
        f.write(template)

    print(" Created website integration template")
    print(" File: website_integration_template.py")

# Run export functions
print(" EXPORTING RESULTS")
print("=" * 50)

# Export sample passwords
sample_data = export_sample_passwords(count=10)

# Create integration template
create_integration_template()

print("\n SETUP COMPLETE!")
print("password generator is ready")

 EXPORTING RESULTS
 Loaded comprehensive weak patterns dataset:
   • 70 common passwords
   • 54 dictionary words
   • 48 keyboard patterns
   • 12 regex patterns
   • 7 date patterns
   • Enhanced l33t speak detection
 Exported 10 sample passwords to sample_passwords.json
 File available for download in Colab Files panel
 Created website integration template
 File: website_integration_template.py

 SETUP COMPLETE!
password generator is ready


In [ ]:
# Cell 9: Final System Verification
def final_system_test():
    print(" FINAL SYSTEM VERIFICATION")
    print("=" * 60)

    try:
        # Test 1: Generator initialization
        generator = GooglePasswordGenerator()
        print(" Generator initialized successfully")

        # Test 2: Password generation
        password = generator.generate_secure_password(length=16)
        print(f" Password generated: {len(password)} characters")

        # Test 3: Analysis
        analysis = generator.analyze_password_strength(password)
        print(f" Analysis complete: {analysis['security_score']}/100 score")

        # Test 4: Dataset loading
        weakness_details = generator.get_detailed_weakness_analysis("password123")
        total_detections = sum(len(v) for v in weakness_details.values())
        print(f" Datasets loaded: {total_detections} weaknesses detected in test")

        # Test 5: API functions
        api = PasswordAPI()
        api_result = api.generate_password_api(length=12)
        print(f" API functions working: {api_result['success']}")

        print("\n ALL SYSTEMS OPERATIONAL!")
        print("Your password generator is production-ready!")

        return True

    except Exception as e:
        print(f" Error during verification: {e}")
        return False

# Run final verification
success = final_system_test()

if success:
    print("Google-compliant password generator is fully operational!")

else:
    print("\n failed")

 FINAL SYSTEM VERIFICATION
 Loaded comprehensive weak patterns dataset:
   • 70 common passwords
   • 54 dictionary words
   • 48 keyboard patterns
   • 12 regex patterns
   • 7 date patterns
   • Enhanced l33t speak detection
 Generator initialized successfully
 Password generated: 16 characters
 Analysis complete: 100/100 score
 Datasets loaded: 6 weaknesses detected in test
 Loaded comprehensive weak patterns dataset:
   • 70 common passwords
   • 54 dictionary words
   • 48 keyboard patterns
   • 12 regex patterns
   • 7 date patterns
   • Enhanced l33t speak detection
 API functions working: True

 ALL SYSTEMS OPERATIONAL!
Your password generator is production-ready!
Google-compliant password generator is fully operational!


In [ ]:
# Cell 9: Final System Verification
def final_system_test():
    print(" FINAL SYSTEM VERIFICATION")
    print("=" * 60)

    try:
        # Test 1: Generator initialization
        generator = GooglePasswordGenerator()
        print(" Generator initialized successfully")

        # Test 2: Password generation
        password = generator.generate_secure_password(length=16)
        print(f" Password generated: {len(password)} characters")

        # Test 3: Analysis
        analysis = generator.analyze_password_strength(password)
        print(f" Analysis complete: {analysis['security_score']}/100 score")

        # Test 4: Dataset loading
        weakness_details = generator.get_detailed_weakness_analysis("password123")
        total_detections = sum(len(v) for v in weakness_details.values())
        print(f" Datasets loaded: {total_detections} weaknesses detected in test")

        # Test 5: API functions
        api = PasswordAPI()
        api_result = api.generate_password_api(length=12)
        print(f" API functions working: {api_result['success']}")

        print("\n ALL SYSTEMS OPERATIONAL!")
        print("Your password generator is production-ready!")

        return True

    except Exception as e:
        print(f" Error during verification: {e}")
        return False

# Run final verification
success = final_system_test()

if success:
    print("Google-compliant password generator is fully operational!")

else:
    print("\n failed")

import pickle

# Save sample_data as a pickle file
try:
    with open('sample_passwords.pkl', 'wb') as f:
        pickle.dump(sample_data, f)
    print("\n Successfully saved sample_data as sample_passwords.pkl")
except NameError:
    print("\n Error: 'sample_data' variable not found. Please run the cell that generates sample_data first.")
except Exception as e:
    print(f"\n Error saving sample_data as pickle file: {e}")

 FINAL SYSTEM VERIFICATION
 Loaded comprehensive weak patterns dataset:
   • 70 common passwords
   • 54 dictionary words
   • 48 keyboard patterns
   • 12 regex patterns
   • 7 date patterns
   • Enhanced l33t speak detection
 Generator initialized successfully
 Password generated: 16 characters
 Analysis complete: 100/100 score
 Datasets loaded: 6 weaknesses detected in test
 Loaded comprehensive weak patterns dataset:
   • 70 common passwords
   • 54 dictionary words
   • 48 keyboard patterns
   • 12 regex patterns
   • 7 date patterns
   • Enhanced l33t speak detection
 API functions working: True

 ALL SYSTEMS OPERATIONAL!
Your password generator is production-ready!
Google-compliant password generator is fully operational!

 Successfully saved sample_data as sample_passwords.pkl


In [ ]:
from google.colab import files
files.download('sample_passwords.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>